# Create Coincident Load By Load Control Meter
Finds usage for all meters in a load control program during the GRE monthly coincidental load.

In [0]:
%run ../../Utilities/ConfigUtilities

In [0]:
from pyspark.sql.functions import col, concat, lit, sum

debug = 1

In [0]:
# Read the coincidental load information with index information.
coincid_load_df = spark.read.format("delta").load(COINCIDENTAL_LOAD_INDEX_PATH)

if debug:
  display(coincid_load_df)

In [0]:
# Read the data containing the load control meters and associated programs.
load_control_df = spark.read.format("delta").load(LOAD_CONTROL_METER_PROGRAM_MAP_PATH)

if debug:
    print(load_control_df.count())
    display(load_control_df)

In [0]:
display(load_control_df.filter(col('BI_MTR_NBR') == ""))

In [0]:
from pyspark.sql.functions import collect_list, concat_ws

load_control_df = load_control_df.orderBy("BI_MTR_NBR", "BI_LOAD_TYPE")

# Group by BI_METER_NBR and concatenate BI_LOAD_TYPE separated by a comma
load_control_agg_df = load_control_df.groupBy("BI_MTR_NBR").agg(concat_ws(",", collect_list("BI_LOAD_TYPE")).alias("BI_LOAD_TYPE_LIST"), 
                                                      concat_ws(",", collect_list("ResourceName")).alias("Program_List"))

if debug:
    print(load_control_agg_df.count())
    display(load_control_agg_df)
    display(load_control_agg_df.filter(col('BI_MTR_NBR') == '65918693'))

In [0]:
# Read the hourly meter data.
hourly_df = spark.read.format("delta").load(MDM_HOURLY_PATH)

if debug:
    display(hourly_df)

In [0]:
# Filter the hourly data to the coincidental peak hour by joining with the coincidental peak dataframe.
hourly_coincid_df = hourly_df.join(coincid_load_df, hourly_df.EndMeterSampleIndex == coincid_load_df.MeterSampleIndex)

hourly_coincid_df = hourly_coincid_df.drop('MeterSampleIndex', 'LocalYear', 'LocalMonth', 'LocalDay', 'HourEnding')

if debug:
    display(hourly_coincid_df)

In [0]:
# Get totals for the coincidental peak slot for all meters.
hourly_coincid_total_df = hourly_coincid_df.groupBy("CoincidYear", "CoincidMonth").agg(sum("HourlyAMIValue").alias("TotalCPHourlyAMIValue"), sum("HourlyVEEValue").alias("TotalCPHourlyVEEValue"))

hourly_coincid_total_df = hourly_coincid_total_df.withColumn("YearMonth", concat(hourly_coincid_total_df.CoincidYear, lit("-"), hourly_coincid_total_df.CoincidMonth))

if debug:
    display(hourly_coincid_total_df)


In [0]:
# Save this total for comparison with the GRE total and the load control total.
hourly_coincid_total_df.write.format("delta").mode("overwrite").save(COINCIDENTAL_PEAK_ALL_METER_TOTAL_PATH) 


In [0]:
# Clean up the delta history.
spark.sql(f"VACUUM '{COINCIDENTAL_PEAK_ALL_METER_TOTAL_PATH}'")

In [0]:
# Filter to the load control meters.
hourly_coincid_lc_df = hourly_coincid_df.join(load_control_agg_df, (hourly_coincid_df.MeterNumber == load_control_agg_df.BI_MTR_NBR), how='inner')

if debug:
    display(hourly_coincid_lc_df)

In [0]:
hourly_coincid_lc_df = hourly_coincid_lc_df.withColumn("YearMonth", concat(hourly_coincid_lc_df.CoincidYear, lit("-"), hourly_coincid_lc_df.CoincidMonth))

In [0]:
# Save to the gold tier.
hourly_coincid_lc_df.write.format("delta").mode("overwrite").save(COINCIDENTAL_PEAK_USAGE_PATH)

In [0]:
# Clean up the delta history.
spark.sql(f"VACUUM '{COINCIDENTAL_PEAK_USAGE_PATH}'")

In [0]:
# Create a YearMonth dimension table for use in PowerBI.
yearmonth_df = hourly_coincid_lc_df.select("CoincidYear", "CoincidMonth", "YearMonth").distinct()


In [0]:
# Join with the demand rates to have them available to all other datasets in PowerBI.  
demand_rate_df = spark.read.csv(DEMAND_RATE_PATH, header=True, inferSchema=True)

yearmonth_demand_df = yearmonth_df.join(demand_rate_df, yearmonth_df.CoincidMonth == demand_rate_df.Month, how='inner')

if debug:
    display(yearmonth_demand_df)


In [0]:
# Save
yearmonth_demand_df.write.format("delta").mode("overwrite").save(COINCIDENTAL_PEAK_DIM_YEARMONTH_PATH)

In [0]:
# Clean up the delta history.
spark.sql(f"VACUUM '{COINCIDENTAL_PEAK_DIM_YEARMONTH_PATH}'")